In [7]:
import logging
import pandas as pd
import json
import plotly.express as px
import plotly.graph_objects as go
import hashlib
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import random
import sqlite3

def setup_logger(name):
    """Set up a logger for a given module."""
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)

    # Create file handler which logs even debug messages
    fh = logging.FileHandler('app.log')
    formatter = logging.Formatter('%(asctime)s [%(levelname)s] - %(message)s')
    fh.setFormatter(formatter)

    # Add the handler to the logger
    if not logger.handlers:
        logger.addHandler(fh)

    return logger

logger = setup_logger(__name__)

def set_start_end():
    today = datetime.now()
    weekday = today.weekday()  # Monday is 0 and Sunday is 6

    # Calculate the start date (Tuesday)
    if weekday >= 1:  # If today is Tuesday or after
        start_date = today - timedelta(days=(weekday - 1))
    else:  # If today is before Tuesday
        start_date = today - timedelta(days=(weekday + 6))

    # Calculate the end date (Monday)
    if weekday <= 0:  # If today is Monday
        end_date = today
    else:  # If today is after Monday
        end_date = today + timedelta(days=(7 - weekday))

    # Convert datetime to date
    return start_date.date(), end_date.date()

def generate_game_id(row):
    try:
        # Example: Use a combination of date, home team, and away team to generate a unique ID
        identifier = f"{row['date']}_{row['home_team']}_{row['away_team']}"
        return hashlib.md5(identifier.encode()).hexdigest()
    except Exception as e:
        logger.exception("Generate Game error")

# Function to convert the betting odds to integers while handling the signs
def convert_to_int(value):
    try:
        if value == 'EVEN':
            return 0
        if value.startswith('+'):
            return int(value[1:])
        elif value.startswith('-'):
            return int(value)
        else:
            return int(value)
    except Exception as e:
        logger.exception("Convert to int error")
        return -1

def concat_values(x, y, z=None):
    if z:
        return f"{x} {y} {z}"
    return f"{x} {y}"

def get_espn_expert_data():
    try:
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--disable-extensions")
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")  # This line can be important in certain environments
        options.set_capability('goog:loggingPrefs', {'browser': 'SEVERE'})
        # Initialize the Chrome WebDriver with the specified options
        driver = webdriver.Chrome(options=options)
        driver.get("https://www.espn.com/nfl/picks")
        #time.sleep(10)
        driver.implicitly_wait(10)
        # get the HTML source
        html = driver.page_source
        # create a BeautifulSoup object
        soup = BeautifulSoup(html, "html.parser")
        # close the driver
        driver.quit()

        week = soup.find('h1', class_='headline headline__h1 dib').get_text(strip=True).split('- ')[1]

        # Extract game details
        games = []
        game_rows = soup.select('.Table--fixed-left .Table__TBODY .Table__TR')
        for row in game_rows:
            game_info_element = row.select_one('.wrap-competition a')
            game_time_element = row.select_one('.competition-dates')
            if game_info_element and game_time_element:
                game_info = game_info_element.text
                game_time = game_time_element.text
                games.append((game_info, game_time))

        # Extract expert names
        experts = []
        expert_headers = soup.select('.Table__Scroller .Table__THEAD .Table__TH')
        for header in expert_headers:
            expert_name_element = header.select_one('div')
            if expert_name_element:
                expert_name = expert_name_element.text.strip()
                experts.append(expert_name)

        # Extract picks
        picks = []
        pick_rows = soup.select('.Table__Scroller .Table__TBODY .Table__TR')
        for row in pick_rows:
            pick_row = []
            pick_cells = row.select('.Table__TD')
            for cell in pick_cells:
                team_logo = cell.select_one('img')
                if team_logo:
                    # Extract the team abbreviation from the image URL
                    team = team_logo['src'].split('/')[-1].split('.')[0]
                else:
                    team = None
                pick_row.append(team)
            picks.append(pick_row)

        # Create DataFrame
        data = {'Game': [game[0] for game in games], 'Time': [game[1] for game in games]}
        for i, expert in enumerate(experts):
            data[expert] = [pick[i] for pick in picks]

        data['Game'].append(None)
        data['Time'].append(None)

        df = pd.DataFrame(data)
        df.dropna(inplace=True)

        df['week'] = week

        convert_dict = {
            "min": "Vikings", "phi": "Eagles", "bal": "Ravens", "det": "Lions", "mia": "Dolphins",
            "nyj": "Jets", "atl": "Falcons", "gb": "Packers", "hou" : "Texans", "lac": "Chargers",
            "buf": "Bills", "den": "Broncos", "kc": "Chiefs", "chi": "Bears", "sf": "49ers", "pit": "Steelers"
        }

        for ix, row in df.iterrows():
            values = row.to_list()[2:]
            values_len = len(values)
            values_dict = {}
            for value in values:
                if value not in values_dict.keys():
                    values_dict[value] = 1
                else:
                    values_dict[value] += 1
            #sorting
            values_dict = dict(sorted(values_dict.items(), key=lambda item: item[1], reverse=True))
            top_key = next(iter(values_dict))
            if top_key in convert_dict:
                converted_key = convert_dict[top_key]
            else:
                converted_key = top_key
            pct = int(values_dict[top_key]/values_len*100)
            message = f"{pct}% of experts chose {converted_key}"
            df.loc[ix, "pct"] = pct
            df.loc[ix, "message"] = message

        return df[["week", "Game", "Time", "message"]]
    except Exception as e:
        logger.exception("get espn data")

def get_data(start_date, end_date):
    try:
        logger.info(f"fetching data for {start_date}-{end_date}")
        # Configure ChromeOptions for headless browsing
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--disable-extensions")
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")  # This line can be important in certain environments
        options.set_capability('goog:loggingPrefs', {'browser': 'SEVERE'})
        # Initialize the Chrome WebDriver with the specified options
        driver = webdriver.Chrome(options=options)
        driver.get("https://www.bovada.lv/sports/football/nfl")
        # wait for the page to load
        time.sleep(10)
        driver.implicitly_wait(10)
        # get the HTML source
        html = driver.page_source
        # create a BeautifulSoup object
        soup = BeautifulSoup(html, "html.parser")
        # close the driver
        driver.quit()

        data = []
        sections = soup.find_all("section", {"class":"coupon-content more-info"})#soup.find_all("section", {"class":"coupon-content more-info"})
        for game in sections:
            try:
                item = str(game).split('>')
                info = [x.split('<')[0].strip() for x in item if not x.startswith("<")]
                data.append(info)
            except Exception as e:
                logger.exception("get data section error")
                pass

        df = pd.DataFrame(data)

        df["Home Spread"] = df.apply(lambda row: concat_values(row[10], row[11]), axis=1)
        df["Away Spread"] = df.apply(lambda row: concat_values(row[12], row[13]), axis=1)
        df["total_home"] = df.apply(lambda row: concat_values(row[16], row[17], row[18]), axis=1)
        df["total_away"] = df.apply(lambda row: concat_values(row[19], row[20], row[21]), axis=1)
        #drop columns
        df.drop(columns = [3, 4, 5, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22], inplace=True)
        columns = ["date", "time", "bets", "home_team", "away_team", "home_win", "away_win", "home_spread", "away_spread", "total_over", "total_under"]
        df.columns = columns

        #remove plus from bets
        df['bets'] = df['bets'].apply(lambda x: x[2:])

        #date operations
        #filter data for date
        if isinstance(start_date, str):
            start_date = datetime.strptime(start_date, '%Y-%m-%d')  # Adjust the format if needed
        if isinstance(end_date, str):
            end_date = datetime.strptime(end_date, '%Y-%m-%d')  # Adjust the format if needed
            # Ensure the 'date' column in df is of type datetime.date
        
        # Ensure the 'date' column in df is of type datetime
        df['date'] = pd.to_datetime(df['date'])

        df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
        #create day of the week column
        df["day"] = df['date'].dt.strftime('%A')
        #set back to string
        df['date'] = df['date'].dt.strftime('%Y-%m-%d')
        df.reset_index(inplace=True, drop=True)

        # Applying the conversion to the 'win_home' and "Away Win" columns
        df['home_win'] = df['home_win'].apply(convert_to_int)
        df["away_win"] = df["away_win"].apply(convert_to_int)
        #ranking
        home = df[["home_team", 'home_win']].rename(columns={'home_team': 'team', 'home_win': 'odds'})
        away = df[['away_team', "away_win"]].rename(columns={'away_team': 'team', "away_win": 'odds'})
        combined = pd.concat([home, away]).sort_values('odds', ascending=False)
        combined['index'] = combined.index
        combined.index = range(0, 2*len(combined), 2)
        df['points'] = None
        # Iterating over the combined DataFrame to assign ranks
        for i, x in combined.iterrows():
            df.at[x['index'], 'points'] = (i-len(combined))/2
        current_df = df.sort_values('points', ascending=False)
        #add game id
        current_df["game_id"] = current_df.apply(generate_game_id, axis=1)
        #change column order
        current_df = current_df[['date', 'day', 'time', 'bets', 'home_team', 'away_team', 'points', 'home_win', 'away_win', 'home_spread', 'away_spread', 'total_over', 'total_under', 'game_id']]
        log_data = current_df[['game_id', 'date', 'home_team', 'away_team', 'home_win', 'away_win', 'points']]

        return current_df
    except Exception as e:
        logger.exception("get data")

def load_historical_data(start_date, end_date):
    try:
        # Connect to SQLite database
        conn = sqlite3.connect('data-log.db')

        # Convert start and end dates to datetime objects if they are not
        if not isinstance(start_date, datetime):
            start_date = datetime.strptime(start_date, '%Y-%m-%d')
        if not isinstance(end_date, datetime):
            end_date = datetime.strptime(end_date, '%Y-%m-%d')

        # Format dates for SQL query
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')

        # SQL query to load data between start_date and end_date
        query = f'''
        SELECT datetime, game_id, date, home_team, away_team, home_win, away_win, points 
        FROM nfl_data 
        WHERE datetime BETWEEN '{start_date_str}' AND '{end_date_str}'
        '''

        # Execute query and fetch data
        df = pd.read_sql_query(query, conn)
        conn.close()

        # Process data to determine which team has the lower win odds
        plot_data = []
        for _, row in df.iterrows():
            home_team = row['home_team']
            away_team = row['away_team']
            home_win = row['home_win']
            away_win = row['away_win']
            points = row['points']

            if home_win < away_win:
                home_points = points
                away_points = -points
            else:
                home_points = -points
                away_points = points

            plot_data.append({
                'DateTime': row['datetime'],
                'Team': home_team,
                'Win': home_win,
                'Type': 'Home Win',
                'points': home_points
            })
            plot_data.append({
                'DateTime': row['datetime'],
                'Team': away_team,
                'Win': away_win,
                'Type': 'Away Win',
                'points': away_points
            })

        return pd.DataFrame(plot_data)
    except Exception as e:
        logger.exception("get historical data")

In [13]:
load_historical_data(start_date, end_date)

In [6]:
get_espn_expert_data()

,week,Game,Time,message
0,Week 8,NO at LAC,Sun 4:05PM,91% of experts chose Chargers
1,Week 8,BUF at SEA,Sun 4:05PM,75% of experts chose Bills
2,Week 8,CAR at DEN,Sun 4:25PM,83% of experts chose Broncos
3,Week 8,KC at LV,Sun 4:25PM,91% of experts chose Chiefs
4,Week 8,CHI at WSH,Sun 4:25PM,66% of experts chose Bears
5,Week 8,PHI at CIN,Sun 1:00PM,66% of experts chose Eagles
6,Week 8,BAL at CLE,Sun 1:00PM,91% of experts chose Ravens
7,Week 8,TEN at DET,Sun 1:00PM,91% of experts chose Lions
8,Week 8,ARI at MIA,Sun 1:00PM,66% of experts chose Dolphins
9,Week 8,NYJ at NE,Sun 1:00PM,83% of experts chose Jets


In [8]:
start_date, end_date = set_start_end()
get_data(start_date, end_date)

In [11]:
logger.info(f"fetching data for {start_date}-{end_date}")
# Configure ChromeOptions for headless browsing
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")  # This line can be important in certain environments
options.set_capability('goog:loggingPrefs', {'browser': 'SEVERE'})
# Initialize the Chrome WebDriver with the specified options
driver = webdriver.Chrome(options=options)
driver.get("https://www.bovada.lv/sports/football/nfl")
# wait for the page to load
time.sleep(10)
driver.implicitly_wait(10)
# get the HTML source
html = driver.page_source
# create a BeautifulSoup object
soup = BeautifulSoup(html, "html.parser")
# close the driver
driver.quit()

data = []
sections = soup.find_all("section", {"class":"coupon-content more-info"})#soup.find_all("section", {"class":"coupon-content more-info"})
for game in sections:
    try:
        item = str(game).split('>')
        info = [x.split('<')[0].strip() for x in item if not x.startswith("<")]
        data.append(info)
    except Exception as e:
        logger.exception("get data section error")
        pass

df = pd.DataFrame(data)

In [12]:
df

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,HT,14,3,+ 23,Bets,HT,14,3,Buffalo Bills,Seattle Seahawks,...,+425,O,38.0,(-120),U,38.0,(-110),,None,None
1,2Q,1:37,2,9,+ 70,Bets,2Q,1:37,2,9,...,(-120),+425,-700,O,32.5,(-115),U,32.5,(-115),
2,2Q,3:25,7,14,+ 24,Bets,2Q,3:25,7,14,...,(-110),+750,-1600,O,47.0,(-115),U,47.0,(-115),
3,2Q,5:24,0,6,+ 11,Bets,2Q,5:24,0,6,...,(-115),+350,-550,O,39.5,(-120),U,39.5,(-110),
4,2Q,1:09,14,10,+ 38,Bets,2Q,1:09,14,10,...,(-110),-475,+315,O,45.5,(-115),U,45.5,(-115),
5,Second Half,Second Half,Buffalo Bills,Seattle Seahawks,+ 23,Bets,+0.5,(-105),-0.5,(-115),...,(-105),,None,None,None,None,None,None,None,None
6,10/27/24,8:20 PM,+ 292,Bets,10/27/24,8:20 PM,Dallas Cowboys,San Francisco 49ers,+ 292,Bets,...,47.5,(-110),U,47.5,(-110),,None,None,None,None
7,10/28/24,8:15 PM,+ 255,Bets,10/28/24,8:15 PM,New York Giants,Pittsburgh Steelers,+ 255,Bets,...,36.5,(-105),U,36.5,(-115),,None,None,None,None


In [9]:
logger.info(f"fetching data for {start_date}-{end_date}")
# Configure ChromeOptions for headless browsing
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")  # This line can be important in certain environments
options.set_capability('goog:loggingPrefs', {'browser': 'SEVERE'})
# Initialize the Chrome WebDriver with the specified options
driver = webdriver.Chrome(options=options)
driver.get("https://www.bovada.lv/sports/football/nfl")
# wait for the page to load
time.sleep(10)
driver.implicitly_wait(10)
# get the HTML source
html = driver.page_source
# create a BeautifulSoup object
soup = BeautifulSoup(html, "html.parser")
# close the driver
driver.quit()

data = []
sections = soup.find_all("section", {"class":"coupon-content more-info"})#soup.find_all("section", {"class":"coupon-content more-info"})
for game in sections:
    try:
        item = str(game).split('>')
        info = [x.split('<')[0].strip() for x in item if not x.startswith("<")]
        data.append(info)
    except Exception as e:
        logger.exception("get data section error")
        pass

df = pd.DataFrame(data)

df["Home Spread"] = df.apply(lambda row: concat_values(row[10], row[11]), axis=1)
df["Away Spread"] = df.apply(lambda row: concat_values(row[12], row[13]), axis=1)
df["total_home"] = df.apply(lambda row: concat_values(row[16], row[17], row[18]), axis=1)
df["total_away"] = df.apply(lambda row: concat_values(row[19], row[20], row[21]), axis=1)
#drop columns
df.drop(columns = [3, 4, 5, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22], inplace=True)
columns = ["date", "time", "bets", "home_team", "away_team", "home_win", "away_win", "home_spread", "away_spread", "total_over", "total_under"]
df.columns = columns

#remove plus from bets
df['bets'] = df['bets'].apply(lambda x: x[2:])

#date operations
#filter data for date
if isinstance(start_date, str):
    start_date = datetime.strptime(start_date, '%Y-%m-%d')  # Adjust the format if needed
if isinstance(end_date, str):
    end_date = datetime.strptime(end_date, '%Y-%m-%d')  # Adjust the format if needed
    # Ensure the 'date' column in df is of type datetime.date

# Ensure the 'date' column in df is of type datetime
df['date'] = pd.to_datetime(df['date'])

df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
#create day of the week column
df["day"] = df['date'].dt.strftime('%A')
#set back to string
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
df.reset_index(inplace=True, drop=True)

# Applying the conversion to the 'win_home' and "Away Win" columns
df['home_win'] = df['home_win'].apply(convert_to_int)
df["away_win"] = df["away_win"].apply(convert_to_int)
#ranking
home = df[["home_team", 'home_win']].rename(columns={'home_team': 'team', 'home_win': 'odds'})
away = df[['away_team', "away_win"]].rename(columns={'away_team': 'team', "away_win": 'odds'})
combined = pd.concat([home, away]).sort_values('odds', ascending=False)
combined['index'] = combined.index
combined.index = range(0, 2*len(combined), 2)
df['points'] = None
# Iterating over the combined DataFrame to assign ranks
for i, x in combined.iterrows():
    df.at[x['index'], 'points'] = (i-len(combined))/2
current_df = df.sort_values('points', ascending=False)
#add game id
current_df["game_id"] = current_df.apply(generate_game_id, axis=1)
#change column order
current_df = current_df[['date', 'day', 'time', 'bets', 'home_team', 'away_team', 'points', 'home_win', 'away_win', 'home_spread', 'away_spread', 'total_over', 'total_under', 'game_id']]
log_data = current_df[['game_id', 'date', 'home_team', 'away_team', 'home_win', 'away_win', 'points']]

ValueError: Length mismatch: Expected axis has 15 elements, new values have 11 elements

In [10]:
df

,0,1,2,6,7,14,15,23,24,25,26,Home Spread,Away Spread,total_home,total_away
0,HT,14,3,14,3,+10.0,(-110),(-110),,None,None,+ 24 Bets,-10.0 (-120),-700 +425 O,38.0 (-120) U
1,2Q,1:36,2,2Q,1:36,+8.0,(-110),U,32.0,(-110),,New Orleans Saints Los Angeles Chargers,+ 9 Bets,-8.0 (-120) +375,-600 O 32.0
2,2Q,3:24,7,2Q,3:24,+16.0,(-120),U,47.5,(-120),,Carolina Panthers Denver Broncos,+ 24 Bets,-16.0 (-110) +750,-1600 O 47.5
3,2Q,5:28,0,2Q,5:28,+8.5,(-115),U,39.5,(-110),,Chicago Bears Washington Commanders,+ 9 Bets,-8.5 (-115) +350,-550 O 39.5
4,2Q,1:50,14,2Q,1:50,-7.0,(-110),U,47.5,(-115),,Kansas City Chiefs Las Vegas Raiders,+ 38 Bets,+7.0 (-120) -410,+280 O 47.5
5,Second Half,Second Half,Buffalo Bills,+1.0,(-115),(-115),U,None,None,None,None,+110 -130,O 21.0,21.0 (-105),None None
6,10/27/24,8:20 PM,+ 292,Dallas Cowboys,San Francisco 49ers,+200,-240,None,None,None,None,+5.5 (-110),-5.5 (-110),O 47.5 (-110),U 47.5 (-110)
7,10/28/24,8:15 PM,+ 255,New York Giants,Pittsburgh Steelers,+215,-260,None,None,None,None,+5.5 (-105),-5.5 (-115),O 36.5 (-105),U 36.5 (-115)
